### CatBoost GBM Classifier Model Training

In [3]:
!pip install catboost


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.2/99.2 MB 8.3 MB/s eta 0:00:00


In [4]:
# Load necessary libraries
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix
from sklearn.impute import SimpleImputer
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from catboost import CatBoostClassifier

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
df = pd.read_csv("/content/drive/MyDrive/Road_Accident_Aalysis_project/Accidents_Cleaned.csv")
# df = df.sample(100000, random_state=42)


target = 'Severity'
X = df.drop(columns=target)
y = df[target]

In [6]:
categorical_cols = X.select_dtypes(include='object').columns.tolist()
numerical_cols = X.select_dtypes(include=['int64', 'float64', 'bool']).columns.tolist()

In [7]:
numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())
])

categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='constant', fill_value='missing')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numerical_cols),
        ('cat', categorical_transformer, categorical_cols)
    ]
)

In [8]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=42, stratify=y
)

In [11]:
clf_catboost = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('classifier', CatBoostClassifier(
        iterations=100,
        learning_rate=0.05,
        depth=6,
        loss_function='MultiClass',
        eval_metric='Accuracy',
        random_seed=42,
        verbose=50,
        early_stopping_rounds=20,

    ))
])

clf_catboost.fit(X_train[:50000], y_train[:50000])

0:	learn: 0.6557400	total: 1.25s	remaining: 2m 3s
50:	learn: 0.7040600	total: 37.5s	remaining: 36.1s
99:	learn: 0.7230200	total: 1m 13s	remaining: 0us


Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='median')),
                                                                  ('scaler',
                                                                   StandardScaler())]),
                                                  ['Start_Lat', 'Start_Lng',
                                                   'Distance(mi)',
                                                   'Temperature(F)',
                                                   'Wind_Chill(F)',
                                                   'Humidity(%)',
                                                   'Pressure(in)',
                                                   'Visibility(mi)',
                                                   'Wind_Speed(mph)',
                                                   'Precipitation(in)',
                                                   'Amenity', 'Bump',
                                                   'Crossing', 'Gi...
                                                   'IsWeekend', 'IsDay']),
                                                 ('cat',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(fill_value='missing',
                                                                                 strategy='constant')),
                                                                  ('onehot',
                                                                   OneHotEncoder(handle_unknown='ignore'))]),
                                                  ['City', 'County', 'State',
                                                   'Zipcode', 'Country',
                                                   'Timezone', 'Airport_Code',
                                                   'Weather_Timestamp',
                                                   'Wind_Direction',
                                                   'Weather_Condition'])])),
                ('classifier',
                 <catboost.core.CatBoostClassifier object at 0x7ad98efc32f0>)])

In [12]:
y_pred_cb = clf_catboost.predict(X_test)

print('CatBoost Classifier Accuracy:', accuracy_score(y_test, y_pred_cb))
print('\nClassification Report:\n', classification_report(y_test, y_pred_cb))
print('\nConfusion Matrix:\n', confusion_matrix(y_test, y_pred_cb))

CatBoost Classifier Accuracy: 0.7209191310603968

Classification Report:
               precision    recall  f1-score   support

           1       0.00      0.00      0.00      5725
           2       0.73      0.92      0.81    190471
           3       0.68      0.38      0.49     95322
           4       0.00      0.00      0.00      1759

    accuracy                           0.72    293277
   macro avg       0.35      0.33      0.33    293277
weighted avg       0.70      0.72      0.69    293277



/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))



Confusion Matrix:
 [[     0   5558    167      0]
 [     0 174774  15697      0]
 [     0  58667  36655      0]
 [     0    390   1369      0]]
